# Maskinlæring

I denne notebooken skal vi 

- Bruke de nyvaskede og utforskede dataene våre til å trene en eller flere maskinlæringsmodeller.

- Se på hvordan vi kan måle kvaliteten til modellene våre, og hvordan vi kan øke kvalitet. 

- Se på hvordan vi kan kontrollere hvordan modellene våre oppfører seg basert på våre egne prioriteringer.



In [14]:
import pandas as pd

Først leser vi inn dataen vår fra en fil i parquet-format til en Pandas DataFrame:

In [2]:
df = pd.read_parquet("fraud_preprocessed.pq")

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15420 entries, 0 to 15419
Data columns (total 33 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   Month                 15420 non-null  int64
 1   WeekOfMonth           15420 non-null  int64
 2   DayOfWeek             15420 non-null  int64
 3   Make                  15420 non-null  int64
 4   AccidentArea          15420 non-null  int64
 5   DayOfWeekClaimed      15420 non-null  int64
 6   MonthClaimed          15420 non-null  int64
 7   WeekOfMonthClaimed    15420 non-null  int64
 8   Sex                   15420 non-null  int64
 9   MaritalStatus         15420 non-null  int64
 10  Age                   15420 non-null  int64
 11  Fault                 15420 non-null  int64
 12  PolicyType            15420 non-null  int64
 13  VehicleCategory       15420 non-null  int64
 14  VehiclePrice          15420 non-null  int64
 15  FraudFound_P          15420 non-null  int64
 16  Poli

Denne parquet-filen skal du ha lagret fra den andre notebooken hvor du utforsket og vasket dataene. Den skal nå være klar til bruk i maskinlæringsmodellen(e) dine.

Hvis du ønsker kan du dobbelsjekke at alt ser ut til å være på plass slik du forventer, ved å bruke noen av de følgende pandas-funksjonene:

```
df.head()  # Se de første 5 radene
df.info()  # Se datatyper og hvor mange ikke-null verdier det er for hver kolonne
df.describe()  # Se statistisk fordeling av verdier i hver kolonne
for col in df:
    print(df[col].value_counts())  # Tell hvor mange ganger hver unike verdi forekommer
    print(str(len(df[col].unique())) + " unique values in column")
    print()
```

Når man trener en maskinlæringsmodell er det god praksis å sette tilside noe data som man ikke trener på, så man kan teste på den hvor godt modellen din generaliserer.

In [16]:
from sklearn.model_selection import train_test_split

In [21]:
y_col_name = "FraudFound_P"
y = df[y_col_name]
X = df.drop(y_col_name, axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Som så kan man splitte dataen én gang mellom trening og test. Men det kan være at akkurat denne splitten forårsaker unøyaktig trening eller test på grunn av arbitrære mønstre eller støy i dataen som ikke er representative for dataen som helhet. Hvis man ønsker seg et godt mål på hvor bra treningsregimet ditt er kan det lønne seg å splitte på flere forskjellige måter, og sammenligne scorene dine fra å trene og teste på forskjellige splitter.

En metode for dette kalles K-fold cross-validation. Navnet lyder slik fordi man folder dataen i K deler, og for hver av de K delene trener man først på all dataen som ikke er i delen, og så tester man på den delen. Størrelsen på K avgjør også størrelsen på test-settene, som får størrelsen 1/K. K bør være høy nok til at treningssettet inneholder et representativt utvalg av dataen i alle tilfeller, men des høyere K er des mindre representativt for populasjonen vil testfolden være.

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
 
k = 5
k_fold_x_validation = KFold(n_splits=k, random_state=None)
logistic_regressor = LogisticRegression(solver='liblinear')
 
result = cross_val_score(logistic_regressor, X, y, cv=k_fold_x_validation)
 
print("Avg accuracy: {}".format(result.mean()))

Avg accuracy: 0.9401426718547341


Wow, det var en høy nøyaktighet! Over 90% riktig! Pack it up, our work here is done 😎

Eller?

Hva er egentlig nøyaktighet (accuracy), og hva er en "god" nøyaktighet?

Accuracy er hvor mange prosent av klassifikasjonene dine som ble riktig. Hvis du har et binært klassifiseringsproblem slik vi har her, og fordelingen mellom de to klassene ("muffins", "Ikke muffins") i dataen din er 10/90, så vil det å alltid gjette "ikke muffins" gi deg en nøyaktighet på 90%!

Så hvis målet er å identifisere noe muffins, så er ikke 90% nøyaktighet bra i det hele tatt, i dette tilfellet...

Hvis `n(klasse)` er antall medlemmer i en klasse i datasettet er altså grunnlinjen for nøyaktighet i vår klassifisering `1 - (n(muffins)/n(ikke muffins))`

In [37]:
1 - int(y.value_counts()[1])/int(y.value_counts()[0])

0.9363316548251363

In [ ]:
Nøyaktighet bedre en dette vil